# SVM 多クラス分類，Iris

sklearn.svm.SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html<br>
decision_function_shape{‘ovo’, ‘ovr’}, default='ovr'(recommended)

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

#from mlxtend.plotting import plot_decision_regions #決定境界のプロット
import seaborn as sns

import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

irisデータのうち，sepal（がく）と petal（花びら）の長さ[cm]によるクラス分類を行う。  
iris.dataの0番目と2番目の要素だけを抽出してXの配列(150 x 2)に格納    
クラス(花びらの種類 0:Iris-Setosa, 1:Iris-Versicolour, 2:Iris-Virginica)を y(150 x 1)に格納

#### iris データセットの説明
クラス(花びらの種類 0:Iris-Setosa, 1:Iris-Versicolour, 2:Iris-Virginica)を y(150 x 1)に格納。<br>
特徴量：sepal（がく）と petal（花びら）の長さ[cm]と幅[c]，4種類<br>
下記のirisはBunchクラス。これは，辞書(dict)に属性参照の機能を追加したもので，scikit-learnにおけるデータセットのパッケージに用いられる。また，.DESCRでデータセットの内容と統計量を見ることができる。

In [ ]:
# Load iris data
iris = datasets.load_iris()
print(iris.keys())
#print(iris.DESCR) # description

In [ ]:
print(type(iris))
print(type(iris.target_names))

In [ ]:
# to numpy.ndarray
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
print(X.shape, X_test.shape, X_train.shape)
print(y.shape, y_test.shape, y_train.shape)
CV=3

### Grid searchの実施

In [ ]:
parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, 1.0, 10.0], 
              'gamma':[0.01, 0.1, 1.0, 10.0], 'decision_function_shape': ['ovr','ovo'], 'random_state':[123]}
#parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, 1.0, 10.0], 'gamma':[0.01, 0.1, 1.0, 10.0], 'random_state':[123]}
svm = SVC()
clf = GridSearchCV(svm,  parameters, scoring='accuracy', cv=CV)
clf.fit(X_train, y_train)

In [ ]:
print('Best accuracy =', clf.best_score_)
print(clf.best_params_)

#### 性能評価
トレーニングデータに対して行う

In [ ]:
best_clf = clf.best_estimator_
y_train_pred = best_clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

テストデータに対して行う

In [ ]:
#best_clf = clf.best_estimator_
y_test_pred = best_clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

#### 散布図行列
散布図行列をプロットするために，データをpandas.DataFrameに格納<br>
Choosing Colormaps https://matplotlib.org/2.0.2/users/colormaps.html<br>
#### トレーニングデータ

In [ ]:
df = pd.DataFrame(data=X_train, columns=['sepal length','sepal width','petal length','petal width'])
df['species']=y_train # species - 0:Setosa, 1:Versicolour, 2:Virginica
df.head()
fig = sns.pairplot(df, hue='species', palette='brg')
#fig.savefig('fig_SVM_iris_pairplot_train.pdf')

上のプロットを見て，sepal length vs. petarl width が最も分離しやすそうに見える。<br>
これより，この二つの散布図を用いて考察する。

In [ ]:
plt.scatter(X_train[:,0], X_train[:,3], c=y_train_pred, cmap='brg')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,5), sharex=True, sharey=True)

f0 = ax[0].scatter(X_train[:,0], X_train[:,3], c=y_train,      cmap='brg')
f1 = ax[1].scatter(X_train[:,0], X_train[:,3], c=y_train_pred, cmap='brg')

ax[0].set_xlabel('sepal length[cm]', fontsize=16)
ax[0].set_ylabel('petal width[cm]',  fontsize=16)
ax[1].set_xlabel('sepal length[cm]', fontsize=16)
ax[1].set_ylabel('petal width[cm]',  fontsize=16)

fig.colorbar(f0, ax=ax[0])
fig.colorbar(f1, ax=ax[1])

plt.show()

両方の図を見て，どこが違うかを調べる。

In [ ]:
y_train

In [ ]:
y_train_pred

y_trainとy_train_predの異なる要素番号を抽出する

In [ ]:
comp1 = y_train == y_train_pred
Id1 = np.where(comp1==False)
print(Id1[0])
print(y_train[Id1], y_train_pred[Id1])

上記から，84番目の要素，クラス1をクラス2に誤認識

#### テストデータ

In [ ]:
df = pd.DataFrame(data=X_test, columns=['sepal length','sepal width','petal length','petal width'])
df['species']=y_test # species - 0:Setosa, 1:Versicolour, 2:Virginica
df.head()
fig = sns.pairplot(df, hue='species', palette='brg')
#fig.savefig('fig_SVM_iris_pairplot_test.pdf')

上のプロットを見て，sepal length vs. petarl width が最も分離しやすそうに見える。<br>
これらをそれぞれx軸，y軸として，左に元のテストデータをプロット，右に同じテストデータをプロットするが，<br>
推定した値をカラーに反映させるため，c=y_test_predとした。この二つの散布図を用いて考察する。

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,5), sharex=True, sharey=True)

f0 = ax[0].scatter(X_test[:,0], X_test[:,3], c=y_test,      cmap='brg')
f1 = ax[1].scatter(X_test[:,0], X_test[:,3], c=y_test_pred, cmap='brg')

ax[0].set_xlabel('sepal length[cm]', fontsize=16)
ax[0].set_ylabel('petal width[cm]',  fontsize=16)
ax[1].set_xlabel('sepal length[cm]', fontsize=16)
ax[1].set_ylabel('petal width[cm]',  fontsize=16)

fig.colorbar(f0, ax=ax[0])
fig.colorbar(f1, ax=ax[1])

#fig.savefig('fig_SVM_iris_pairplot_true_pred.pdf')
plt.show()

両方の図を見て，グラフの真ん中あたりに，左は赤点だが，右は緑点の違いがある。そこで，具体的にどの要素が違うかを調べる。

In [ ]:
print(y_test)
print(y_test_pred)

y_testとy_test_predの異なる要素番号を抽出する

In [ ]:
comp2 = y_test == y_test_pred
Id2 = np.where(comp2==False)
print(Id2[0])
print(y_test[Id2], y_test_pred[Id2])

上記から，7番目の要素のクラス1をクラス2に誤認識

In [ ]:
comp1 = y_train == y_train_pred
Id1 = np.where(comp1==False)
print(Id1[0])
print(y_train[Id1], y_train_pred[Id1])